In [ ]:
import numpy as np
import pandas as pd
from pyBKT.models import Model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
model = Model(seed = 60, num_fits = 1)

In [ ]:
model.fetch_dataset("https://raw.githubusercontent.com/lishaparmar13/H5P-Enhanced-BKT/main/Above16-CS.csv",'.')

In [ ]:
df2= pd.read_csv(r"Above16-CS.csv", encoding ='latin')
df2.head(5)

In [ ]:
model.fit(data = df2)

In [ ]:
train_df, test_df = train_test_split(df2, test_size=0.2, random_state=42)

In [ ]:
model.fit(data = train_df)
print("Fitted Skills:\n%s" % '\n'.join(model.coef_.keys()))

In [ ]:
# Evaluate with the default RMSE then specify AUC.
model.fit(data= df2)
training_rmse = model.evaluate(data = df2)
training_auc = model.evaluate(data = df2, metric = 'auc')
print("Training RMSE: %f" % training_rmse)
print("Training AUC: %f" % training_auc)

In [ ]:
#define own metric
def mae(true_vals, pred_vals):
  """ Calculates the mean absolute error. """
  return np.mean(np.abs(true_vals - pred_vals))

training_mae = model.evaluate(data = df2, metric = mae)
print("Training MAE: %f" % training_mae)

In [ ]:
model = Model(seed = 60, num_fits = 1)
model.fit(data = train_df)
preds = model.predict(data = test_df)
preds[['Anon Student Id', 'KC(Default)', 'Correct First Attempt', 
       'correct_predictions', 'state_predictions']].head(10)

In [ ]:
# Let's sanity check that we have only trained on the skills that we 
# specified in the call to fit! Note that while it is possible for a 
# BKT prediction to be 0.5 exactly, it is unlikely.
preds[preds['correct_predictions'] != 0.5]['KC(Default)'].unique()

In [ ]:
model.evaluate(data = df2)

In [ ]:
model.evaluate(data=df2, metric=['rmse','accuracy','auc'])

In [ ]:
# Note that folds is an optional parameter as well as the model 
# variant, seed, and crossvalidated metric.
# By default, we crossvalidate on all skills separately.
model.crossvalidate(data = df2, folds = 5)

In [ ]:
model = Model(seed = 60, num_fits = 1)
# Try this with a different skill or metric by replacing the lines below.
skill = 'Algebra'
metric = 'rmse'

simple_cv = model.crossvalidate(data = df2, skills = skill, 
                                metric = metric)
simple_cv

In [ ]:
model = Model(seed = 60, num_fits = 1)
skill = 'Algebra'

multilearn_cv = model.crossvalidate(data = df2, skills = skill,
                                    multilearn ="Answer Type", forgets = True, 
                                    metric = metric)
multilearn_cv

In [ ]:
model = Model(seed = 60, num_fits = 1)
multilearn_cv = model.crossvalidate(data= df2, skills = skill,
                                    multilearn ="question_id", 
                                    metric = metric)
multilearn_cv

In [ ]:
model = Model(seed = 60, num_fits = 1)
# The multiprior model generates different priors based on the first 
# response of each student.
multiprior_cv = model.crossvalidate(data= df2, skills = skill,
                                    multiprior = True, metric = metric,
                                    folds = 3)

model = Model(seed = 60, num_fits = 1)
multipair_cv = model.crossvalidate(data = df2, skills = skill,
                                   multipair = True, metric = metric,
                                   folds = 3)

pd.concat([multiprior_cv, multipair_cv], axis = 0)

In [ ]:
model = Model(seed = 60, num_fits = 1)
# We combine the fifth parameter, forgets, with the previous multilearn
# and multiguess/slip models for a combo model.
combo_cv = model.crossvalidate(data = df2, skills = skill,
                               forgets = True, multilearn = True, 
                               multigs = True, metric = metric)
combo_cv

In [ ]:
skill = 'Geometry'

model.coef_ = {skill: {'prior': 1e-40}}
model.coef_

In [ ]:
# Train the model with the pre-initialized parameters.
model.fit(data = df2, multigs = True)
low_prior_auc = model.evaluate(data = df2, metric = 'auc')

# We can obtain the prior value by indexing into the model.coef_
# dictionary with the skill and parameter names. 
print("Fitted Prior Value: %f" % model.coef_[skill]['prior'])
print("Training AUC: %f" % low_prior_auc)

In [ ]:
# Initialize the prior to be more reasonable.
model.coef_ = {skill: {'prior': 0.5}}
model.fit(data= df2, multigs = True)
normal_prior_auc = model.evaluate(data= df2, metric = 'auc')

# Print the fitted prior value and RMSE.
print("Fitted Prior Value: %f" % model.coef_[skill]['prior'])
print("Training AUC: %f" % normal_prior_auc)

In [ ]:
model.fit(data=df2, forgets = True, multilearn = True, skills=skill, 
          multigs = True)
model.params()

In [ ]:
model.fit(data=df2, forgets = True, multilearn = True, 
          multigs = True)
model.params()